In [3]:
import os
import pandas as pd
import json
import re

DE_DATASET_NON_NORMALIZED_PATH_test = os.path.join(
    os.path.abspath(''),
    '../data/multi_class_test_set_toy.csv'
)
DE_DATASET_NON_NORMALIZED_PATH_train = os.path.join(
    os.path.abspath(''),
    '../data/multi_class_train_set_toy.csv'
)

products_train = pd.read_csv(DE_DATASET_NON_NORMALIZED_PATH_train, index_col=False)
products_test = pd.read_csv(DE_DATASET_NON_NORMALIZED_PATH_test, index_col=False)

products = pd.concat([products_train, products_test])

In [5]:
print(products[:10])
print(products.iloc[46], '\n\n\n', products['description'][46], '\n\n\n', products['title'][46])

   offer_id category subcategory lang  \
0   1005473      toy        lego   en   
1   1005491      toy        lego   en   
2   1005490      toy        lego   en   
3   1005483      toy        lego   en   
4   1005502      toy        lego   en   
5   1005477      toy        lego   en   
6   1005466      toy        lego   en   
7   1005499      toy        lego   en   
8   1005504      toy        lego   en   
9   1005464      toy        lego   en   

                                               title  \
0  NEW LEGO NINJAGO FIRE FANG SET 70674 QUEEN ASP...   
1        LEGO 70674 Ninjago Fire Fang Set NEW IN BOX   
2     Lego Ninjago 70674 Fire Fang w/ 4 minifigs NEW   
3                       Lego Ninjago Fire Fang 70674   
4                   LEGO Ninjago Fire Fang Set 70674   
5  Lego Ninjago set lot 70674 Fire Fang Snake set...   
6                       LEGO 70674 NINJAGO Fire Fang   
7        LEGO 70674 Ninjago Fire Fang Set New Sealed   
8  LEGO 70674 Ninjago Fire Fang Snake Buildi

In [16]:
def full_data_extraction(description, keyvalues):
    results = {}

    positions = [[description.find(kv['translation']), idx] for idx, kv in enumerate(keyvalues)]
    positions.sort(key=lambda x: x[0])
    positions = [pos for pos in positions if pos[0] != -1]

    n_pos = len(positions)
    for idx in range(n_pos):
        start = positions[idx][0]+len(keyvalues[positions[idx][1]]['translation'])
        if idx < n_pos-1:
            end = positions[idx+1][0]
        else:
            end = len(description)
        results[keyvalues[positions[idx][1]]['label']] = description[start:end].strip()

    return results


keyvalues = [{"label": "condition", "translation": "Condition:"},
             {"label": "model", "translation": "Model Theme:"},
             {"label": "age_level", "translation": "Age Level:"},
             {"label": "piece_count", "translation": "Piece Count:"},
             {"label": "theme", "translation": "Product Line (LEGO Theme):"},
             {"label": "age_range", "translation": "Recommended Age Range:"},
             {"label": "mpn", "translation": "MPN:"},
             {"label": "packaging", "translation": "Packaging:"},
             {"label": "modified", "translation": "Modified Item:"},
             {"label": "material", "translation": "Material:"},
             {"label": "type", "translation": "Type:"},
             {"label": "brand", "translation": "Brand:"},
             {"label": "year", "translation": "Year:"},
             {"label": "set_number", "translation": "LEGO Set Number:"},
             {"label": "set_name", "translation": "LEGO Set Name:"},
             {"label": "features", "translation": "Features:"},
             {"label": "character_family", "translation": "Character Family:"},
             {"label": "theme", "translation": "LEGO Theme:"},
             {"label": "character", "translation": "LEGO Character:"},
             {"label": "upc", "translation": "UPC:"}
             ]
ex_keyvalues = full_data_extraction(ex_description, keyvalues)

print(ex_keyvalues)

{'condition': "New: A brand-new  unused  unopened  undamaged item (including handmade items). See the seller's listing for full details. See all condition definitions- opens in a new window or tab ... Read moreabout the condition", 'brand': 'LEGO', 'character': 'Millennium Falcon', 'theme': 'Star Wars', 'age_level': '16+', 'set_number': '75192', 'set_name': 'Millennium Falcon', 'mpn': '75192', 'character_family': 'Star Wars', 'upc': '673419267656'}


In [4]:
# eans = set()
# print(len(products))
# for product in products:
#     print(products['identifiers'][0]['EAN'])
#     eans.add(product['identifiers'][0]['EAN'][0])
#
# print(len(eans))

chemia_count = sum([product['category'] == 'chemia' for product in products])
print(chemia_count)

napoje_count = sum([product['category'] == 'napoje' for product in products])
print(napoje_count)


TypeError: string indices must be integers

In [ ]:
def find_position(searched, search):
    pos = searched.find(search)
    if pos != -1:
        pos = searched[:pos].count(' ')
    return pos

brand_in_title = sum([product['brand'].lower() in product['title'].lower() for product in products])
print(brand_in_title)

position_of_brand = {}
for product in products:
    if not product['brand'].strip():
        continue
    pos = find_position(product['title'].lower(), product['brand'].lower())
    if pos == 17:
        print(product['brand'])
        print(product['title'])
        print('####')
    if pos not in position_of_brand:
        position_of_brand[pos] = 1
    else:
        position_of_brand[pos] += 1

print(position_of_brand)

In [ ]:
import matplotlib.pyplot as plt

counts = {}
# print(products[0])
for product in products:
#     print(product['keyValuePairs']['extras'])
    # if product['keyValuePairs']['origin'].strip() and product['keyValuePairs']['origin'].lower() in product['title'].lower():
    #     print('abc')
    count = product['title'].count(' ')+1
    if count == 17:
        print(product['title'])
    counts[count] = counts.get(count, 0) + 1

plt.bar(counts.keys(), counts.values(), color='g', tick_label=list(counts.keys()))
plt.title(f'Distribution of title lengths')
plt.xlabel('length')
plt.ylabel('count')
plt.savefig(f'pl_wdc_title_length_distribution.png')
plt.show()


In [ ]:
keywords_counts = {}
for product in products:
    for k, v in product['keyValuePairs'].items():
        if v.strip():
            if k in keywords_counts:
                keywords_counts[k] += 1
            else:
                keywords_counts[k] = 1

print(keywords_counts)


def coverage_of_kv(products, kv):
    pos_dict = {}
    for i in range(19):
        pos_dict[i] = 0
    for product in products:
        if kv in product['keyValuePairs'] and product['keyValuePairs'][kv].strip():
            # print(product['keyValuePairs'][kv])
            pos = find_position(product['title'].lower(), product['keyValuePairs'][kv].lower())
            if pos != -1:
                pos = product['title'].count(' ') - (pos+product['keyValuePairs'][kv].count(' '))
#             if pos > 2:
#                 print(pos, product['title'].count(' '), product['keyValuePairs'][kv].count(' '))
#                 print(product['title'])
            pos_dict[pos] = pos_dict.get(pos, 0) + 1
            # if pos != -1:
            #     pos_dict['full'] += 1
    return pos_dict

# kv_cov = coverage_of_kv(products, 'amount')
# plt.bar(kv_cov.keys(), kv_cov.values(), color='g', tick_label=list(kv_cov.keys()))
# plt.title('Distribution of found amount')
# plt.xlabel('position')
# plt.ylabel('count')


for kv in ['amount', 'capacity', 'storage']:
    kv_cov = coverage_of_kv(products, kv)
    plt.bar(kv_cov.keys(), kv_cov.values(), color='g', tick_label=list(kv_cov.keys()))
    print(f'full sum for {kv}: {sum([x for k,x in kv_cov.items() if k != -1])}')
    plt.title(f'Distribution of found {kv}')
    plt.xlabel('position')
    plt.ylabel('count')
    plt.savefig(f'pl_wdc_{kv}_distribution.png')
    plt.show()

In [ ]:
# for product in products:
#     print(product['keyValuePairs']['amount'],
#           '#',
#           product['keyValuePairs']['capacity'])

In [ ]:
c = 0
ca = 0
cc = 0
for product in products:
    amount = product['keyValuePairs']['amount'].strip().lower()
    capacity = product['keyValuePairs']['capacity'].strip().lower()
    if amount:
        ca += 1
    if capacity:
        cc += 1
    if amount and capacity and amount == capacity:
        c += 1

print(c)

In [ ]:
def lengths_of_kv(products, kv):
    lengths = {}
    for product in products:
        if kv in product['keyValuePairs'] and product['keyValuePairs'][kv].strip():
            pos = find_position(product['title'], product['keyValuePairs'][kv])
            kv_len = product['keyValuePairs'][kv].count(' ')+1
            if pos != -1:
                print(pos)
                print(product['keyValuePairs'][kv])
                print(product['title'])
                print('###')
                lengths[kv_len] = lengths.get(kv_len, 0) + 1
    return lengths

amounts_lengths = lengths_of_kv(products, 'amount')
print(amounts_lengths)

In [ ]:
amounts_set = set()
for product in products:
    amounts_set.add(product['keyValuePairs']['amount'].strip())

print(amounts_set)

In [ ]:
import re


def find_units(title, data_tags, reg):
    title = title
    p = re.compile(reg)
    for m in p.finditer(title):
        b_pos = title[:m.span()[0]].count(' ')
        i_pos_end = title[:m.span()[1]].count(' ')
        print(b_pos, i_pos_end)
        i_pos = list(range(b_pos+1, i_pos_end+1))
        first_position = title[:m.start()].count(' ')
        data_tags[b_pos] = 'B-ORG'
        for pos in i_pos:
            data_tags[pos] = 'I-ORG'
        
regex1 = r'([0-9]*[.,])?[0-9]+[ ]{0,}(g|mg|l|ml|kg|L|szt|szt\.|sztuk|litrów|litrow|gram|gramów|miligramów|miligramow|miligram|kilogram|kilogramów|litres|kilograms|miligrams|grams)+'
regex2 = r'\d+[ ]{0,}x[ ]{0,}([0-9]*[.,])?[0-9]+[ ]{0,}(g|mg|l|ml|kg|L|szt|szt\.|sztuk|litrów|litrow|gram|gramów|miligramów|miligramow|miligram|kilogram|kilogramów|litres|kilograms|miligrams|grams)+'

matches = 0
for amount in amounts_set:
    if not(re.match(regex1, amount.lower()) or re.match(regex2, amount.lower())):
        print(amount)
    

title = 'abc def brand 30 x 8 g'
tags = ['O'] * (title.count(' ')+1)
print(title, tags)

find_units(title, tags, regex1)
find_units(title, tags, regex2)
print(title, tags)

In [ ]:
def find_entity_by_match(product, element, tag_b, tag_i, data_tags):
    data_sentence = str(product['title'])
    brand = element
    pos = data_sentence.find(brand)
    n_brand = brand.count(' ') + 1
    if pos > -1:
        first_position = data_sentence[:pos].count(' ')
        for i_brand in range(n_brand):
            tag = tag_b
            if i_brand > 0:
                tag = tag_i
            data_tags[first_position+i_brand] = tag


In [ ]:
ner_data_sentences = []
ner_data_tags = []

ids = []
for idx, product in enumerate(products):
    title = product['title']
    title_split = product['title'].split()

    n_title = len(title_split)
    product_ner_data_tags = ['O'] * n_title

    find_entity_by_match(product, product['brand'], 'B-PER', 'I-PER', product_ner_data_tags)
    
    find_entity_by_match(product, product['keyValuePairs']['amount'], 'B-ORG', 'I-ORG', product_ner_data_tags)
    
    find_units(product['title'].lower(), )
    

    ner_data_sentences.append(title_split)
    ner_data_tags.append(product_ner_data_tags)


print(len(ner_data_tags))

In [ ]:
output_count = len(ner_data_sentences)

RESULTS_PATH = os.path.join(os.path.abspath(''), "../ner_dataset_WDC")

parts = [
    {"label": "train", "start": 0.0, "end": 0.8},
    {"label": "test", "start": 0.8, "end": 1.0},
    {"label": "valid", "start": 0.8, "end": 1.0}
]

for part in parts:
    part_path = os.path.join(RESULTS_PATH, f"{part['label']}.txt")
    start = int(output_count * part['start'])
    end = int(output_count * part['end'])

    result_file = open(part_path, 'w+')

    for idx in range(start, end):
        data_sentence = ner_data_sentences[idx]
        data_tags = ner_data_tags[idx]
        for token, tag in zip(data_sentence, data_tags):
            result_file.write(f'{token}\t{tag}\n')
        result_file.write('\n')
    result_file.close()
